In [1]:
import momepy as mm
import numpy as np
import pandas as pd
from libpysal.graph import Graph
import geopandas as gpd
import pytest
import glob
import shapely
import gc
from shapely import coverage_simplify
import datetime
from libpysal.graph import read_parquet

regions_datadir = '/data/uscuni-ulce/'
data_dir = '/data/uscuni-ulce/processed_data/'
eubucco_files = glob.glob(regions_datadir + 'eubucco_raw/*')
graph_dir = data_dir + 'neigh_graphs/'

In [2]:
region_hulls = gpd.read_parquet(regions_datadir + 'regions/' + 'regions_hull.parquet')

In [3]:
# 12199 - hills, small test
# 69300 - prague medium
# 226 - germany somewhere, largest cluster 
# 106928 + 1 - big one in poland
for region_id, region_hull in region_hulls.iterrows():

    if region_id < 106928: continue
    break
region_id

106928

In [4]:
%%time
buildings = gpd.read_parquet(data_dir + f'/buildings/buildings_{region_id}.parquet')

CPU times: user 125 ms, sys: 59.8 ms, total: 184 ms
Wall time: 183 ms


In [5]:
%%time
buff_lim = mm.buffered_limit(buildings, 
                      'adaptive')

CPU times: user 5.26 s, sys: 114 ms, total: 5.38 s
Wall time: 5.38 s


In [4]:
tessellation = gpd.read_parquet(data_dir + f'/tessellations/tessellation_{region_id}.parquet')

In [5]:
blo_q1 = read_parquet(graph_dir + f'enclosure_graph_{region_id}_knn1.parquet')

In [6]:
blo_q2 = blo_q1.higher_order(k=2, lower_order=True, diagonal=True)

In [7]:
blo_q3 = blo_q1.higher_order(k=3, lower_order=True, diagonal=True)

In [8]:
import scipy as sp

In [9]:
base = blo_q1.sparse.copy()

In [103]:
source = blo_q1.unique_ids[:100]
k = 2

In [104]:
base = blo_q1.sparse.copy()
lil = base.tolil(copy=False)

for i in range(2, k + 1):
    res = base[source, :] @ base
    base = base.tolil(copy=False)
    base[source] = res 
    base = base.tocsr(copy=False)
    # padding = sp.sparse.csr_matrix((base.shape[0] - res.shape[0], base.shape[1]))
    # res = sp.sparse.vstack((res, padding))
    # base = base + res

In [105]:
wtk = base[source, :].tocoo(copy=False)

In [106]:
indices = pd.DataFrame({'neighbor': wtk.col, 'focal':wtk.row}).sort_values(['focal', 'neighbor'])
indices = indices.set_index('focal')['neighbor']

In [108]:
for s in source:
    assert (indices[s] == blo_q2[s].index.values).all()

In [258]:
(blo_q2[0].index.values == wtk.row).all()

ValueError: operands could not be broadcast together with shapes (39,) (71,) 

In [259]:
(blo_q3[0].index.values == wtk.row).all()

ValueError: operands could not be broadcast together with shapes (70,) (71,) 

<2910x2910 sparse array of type '<class 'numpy.float64'>'
	with 71 stored elements in COOrdinate format>

In [229]:
vals = tessellation[tessellation.index>-1].groupby('eID').count()
weights = pd.Series(0, index=blo_q1.unique_ids) ## min should be 1 otherwise i get infinite loops
weights[vals.index.values] = vals.values[:, 0]

In [230]:
dens = (weights / enclosures.area).sort_values(ascending=False)

In [231]:
weights = weights.iloc[dens.index.values]

In [235]:
from collections import deque
import heapq

In [253]:
def process_subgraph(start, graph, weights, dens, visited, max_limit=8000):

    stack = []
    heapq.heappush(stack, (1, -dens.loc[start], start))
    current_weight = weights[start]
    current_list = []
    lists = []
    
    while len(stack):
        
        i, density, current = heapq.heappop(stack)
        weight = weights.loc[current]

        # assert current_weight <= max_limit
        
        # the index has not been processed and there is space in the current partition
        if (current not in visited) and (current_weight + weight <= max_limit):
            
            current_weight += weight
            current_list.append(current)
            visited.add(current)
            
            for v in graph[current].index:
                if v not in visited:
                    heapq.heappush(stack, (i+1, -dens.loc[v], v))
        
        # the index has not been processed, but the current list has reached max capacity
        elif (current not in visited) and (current_weight + weight > max_limit):
            # save the partition order and reset everything
            lists.append(current_list)
            
            current_weight = 0
            current_list = []
            stack = []
            ### reinsert into stack
            heapq.heappush(stack, (i, -density, current))
        
        # if the current index has been processed continue
        else:
            continue
    
    ### the graph disconnects
    lists.append(current_list)
    return lists, visited

In [254]:
start_time = np.datetime64('now')
start_time

numpy.datetime64('2024-05-31T16:05:37')

In [270]:
%%time
i = 0
groups = []
visited = set()
for start in weights.index:

    if start not in visited:
        sub_list, visited = process_subgraph(start, blo_q1, weights, dens, visited, max_limit=100_000)
        for l in sub_list: groups.append(l)

CPU times: user 27.5 s, sys: 12 ms, total: 27.5 s
Wall time: 27.5 s


In [271]:
end = np.datetime64('now')

In [272]:
start_time-end

numpy.timedelta64(-237,'s')

In [273]:
labels = pd.Series(-1, index=weights.index, name='labels')
for cluster_label, locs in enumerate(groups):
    labels[locs] = cluster_label

In [274]:
# labels.to_frame().to_parquet('data/largest_groups.parquet')

In [275]:
_, cnts = np.unique(np.concatenate(groups), return_counts=True)
assert (cnts == 1).all()

In [276]:
assert not (labels==-1).any()

In [277]:
labels.value_counts()

labels
9       36357
18      15440
5       15128
12      14754
29      13708
        ...  
8196        1
8197        1
8198        1
8199        1
8472        1
Name: count, Length: 16945, dtype: int64

In [278]:
weights.groupby(labels).sum().sort_values(ascending=False).iloc[:20]

labels
15    99998
19    99998
2     99997
30    99996
26    99996
29    99995
9     99992
11    99992
14    99983
22    99978
10    99974
21    99965
25    99958
3     99953
8     99951
28    99946
5     99943
18    99937
0     99896
16    99883
dtype: int64

In [279]:
weights.max()

704

In [280]:
enclosures = gpd.read_parquet(data_dir + f'/enclosures/enclosure_{region_id}.parquet')

In [281]:
partitions = enclosures.dissolve(labels)
simplified = coverage_simplify(partitions.geometry, .1)


In [293]:
parititons = gpd.GeoDataFrame({'geometry': simplified.geoms}, index = partitions.index, crs=partitions.crs)

In [295]:
# partitions.loc[weights.groupby(labels).sum().sort_values(ascending=False).iloc[:30].index].explore()

In [306]:
test_enclosures = enclosures.sort_values('geometry')

In [307]:
test_enclosures['part_id'] = -1

In [314]:
part_index = []
npartitions = 100
step = int(enclosures.shape[0] / 100)
new_partitions = []
for start in range(0, enclosures.shape[0], step):
    end = min (start + step, enclosures.shape[0])
    test_enclosures.iloc[start:end, -1] = start

In [315]:
r = test_enclosures.dissolve(by='part_id')

In [318]:
# r.explore()

In [ ]:
res = blo_q1.higher_order(k=5, lower_order=True).assign_self_weight()

In [ ]:
res.to_parquet('data/enclusres_knn3.parquet')

In [ ]:
res.adjacency.shape

In [43]:
%%time
min_buffer: float = 0
max_buffer: float = 100

gabriel = Graph.build_triangulation(buildings.representative_point(), "gabriel", kernel="identity")
max_dist = gabriel.aggregate("max")
buffer = np.clip(max_dist / 2 + max_dist * 0.1, min_buffer, max_buffer).values
buffered_buildings = buildings.buffer(buffer, resolution=2).union_all()

CPU times: user 4.29 s, sys: 36.1 ms, total: 4.33 s
Wall time: 4.32 s


In [44]:
res = gpd.GeoDataFrame({'geometry': [geom for geom in buffered_buildings.geoms]}, crs=buildings.crs)

In [45]:
res.to_parquet('data/gabriel_clusters.parquet')

In [47]:
# res.explore()